In [2]:
from scripts.round3.figures_import_helper_r3 import *

# illustrating impact of different IC constraints, using city data

print('data located at \n\t{}'.format(path_data))
print('plots saved at \n\t{}\n'.format(path_figures))

print('=' * 20 + '\n' + 'Tsuchiura (before min. price)')
print('collecting and processing data')
tsuchiura_data = auction_data.AuctionData(
    os.path.join(path_data, 'tsuchiura_data.csv'))

data located at 
	/home/sylvain/Dropbox/Econ/papiers/gameTheory/missing_bids/data/data_for_missing_bids_figures
plots saved at 
	/home/sylvain/Dropbox/Econ/papiers/gameTheory/missing_bids/data/data_for_missing_bids_figures/figures/20200719

Tsuchiura (before min. price)
collecting and processing data


In [3]:
tsuchiura_before_min_price_ = auction_data.AuctionData(
    tsuchiura_data.df_bids.loc[tsuchiura_data.data.minprice.isnull()])
tsuchiura_before_min_price = auction_data.AuctionData.from_clean_bids(
    tsuchiura_before_min_price_.df_bids.loc[
        tsuchiura_before_min_price_.df_bids.norm_bid > .9])

In [4]:
filter_ties = auction_data.FilterTies(tolerance=.0001)
tsuchiura_before_min_price_filtered = filter_ties(tsuchiura_before_min_price)

In [5]:
constraints = environments.MarkupConstraint(max_markup=.5, min_markup=.4)
empty_constraints = environments.EmptyConstraint()
moment_matrix = auction_data.moment_matrix(
    analytics.ordered_deviations(all_deviations_tsuchiura), 'slope')
moment_weights = np.identity(len(analytics.ordered_deviations(
    all_deviations_tsuchiura)))

In [6]:
solver = solvers.ParallelSolver(
            data=tsuchiura_before_min_price_filtered,
            deviations=all_deviations_tsuchiura,
            metric=analytics.IsNonCompetitive,
            plausibility_constraints=[constraints],
            num_points=5000,
            seed=1,
            project=False,
            filter_ties=None,
            num_evaluations=100,
            confidence_level=.95,
            moment_matrix=moment_matrix,
            moment_weights=moment_weights
        )
solver.result.solution

0.9551545668871826

In [7]:
solver = solvers.ParallelSolver(
            data=tsuchiura_before_min_price_filtered,
            deviations=all_deviations_tsuchiura,
            metric=analytics.EfficientIsNonCompetitive,
            plausibility_constraints=[empty_constraints],
            num_points=5000,
            seed=1,
            project=False,
            filter_ties=None,
            num_evaluations=100,
            confidence_level=.95,
            moment_matrix=moment_matrix,
            moment_weights=moment_weights
        )
solver.result.solution

0.15565923251380687

In [16]:
analytics.EfficientIsNonCompetitive.min_markup = .01

analytics.EfficientIsNonCompetitive.min_markup

0.01

In [17]:
solver = solvers.ParallelSolver(
            data=tsuchiura_before_min_price_filtered,
            deviations=all_deviations_tsuchiura,
            metric=analytics.EfficientIsNonCompetitive,
            plausibility_constraints=[empty_constraints],
            num_points=5000,
            seed=1,
            project=False,
            filter_ties=None,
            num_evaluations=100,
            confidence_level=.95,
            moment_matrix=moment_matrix,
            moment_weights=moment_weights
        )
solver.result.solution

0.15565923251380687

In [10]:
solver.tolerance

array([[5.40082412e-04],
       [5.05343443e-04],
       [1.00000000e-08]])

In [11]:
solver.get_solver(0).demands

[0.6855541718555417, 0.1980074719800747, 0.1980074719800747]